In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Load the cleaned CSV file
df = pd.read_csv('cleaned_combined_articles.csv')

# Separate the gold label
gold_label = df['gold_label']
Y= gold_label
X = df.drop(columns=['gold_label'])
X = X.drop(columns=['link'])

X['text'] = X['title'] + " " + X['content']
# X = X.drop(columns=['title', 'content'])

print("X: " , X.head())

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# bag of words class to store urdu vocubulary
class BagOfWords:

    def __init__(self):
        self.vocabulary={}

    def fit(self,data):
        
        all_words= []
        thebag= {}
        for line in data:
            words= line.split()
            for word in words:
                if word not in thebag:
                    thebag[word]=1
                else:
                    thebag[word]+=1
        
        self.vocabulary= { word : i for i , word in enumerate(thebag.keys())}
        print(self.vocabulary)

    def vectorize(self, line):
        count_vector=np.zeros(len(self.vocabulary))
        words = line.split()
        for word in words:
            if word in self.vocabulary:
                index = self.vocabulary[word]
                count_vector[index] += 1 
        return count_vector

# print(np.unique(Y_test))
bow = BagOfWords()
bow.fit(X_train['text'])

labels= list(np.unique(Y_test))
print(labels)



X:      id                                              title  \
0  391      پاکستانی طلبا عالمی نیوکلیئر اولمپیاڈ اعزازات   
1   76                              ملک آج ڈالر قیمت رہی   
2  280  امریکی سینیٹ اسرائیل اسلحے فروخت روکنے متعلق  ...   
3   63    پاکستان سعودی عرب درآمدات کمی ایران انحصار بڑھ   
4  114            غزہ اسرائیل رہائشی بمباری  فلسطینی شہید   

                                             content  \
0  پاکستانی طلبا بین الاقوامی نیوکلیئر سائنس اولم...   
1  کراچی ملکی تبادلہ منڈیوں ڈالر قیمت اضافہ ہوگیا...   
2  واشنگٹن امریکی سینیٹ اسرائیل غزہ جنگ اسلحے فرو...   
3  اسلام آباد رواں مالی سال پاکستان سعودی عرب در...   
4  عالمی فوجداری عدالت نیتن یاہو وارنٹ گرفتاری جا...   

                                                text  
0  پاکستانی طلبا عالمی نیوکلیئر اولمپیاڈ اعزازات ...  
1  ملک آج ڈالر قیمت رہی کراچی ملکی تبادلہ منڈیوں...  
2  امریکی سینیٹ اسرائیل اسلحے فروخت روکنے متعلق  ...  
3  پاکستان سعودی عرب درآمدات کمی ایران انحصار بڑ...  
4  غزہ اسرائیل رہ

### Multi-Nomial Naive Bayes


In [2]:

class MultinomialNaiveBayes:
    def __init__(self):
        self.classes = None
        self.class_priors = None
        self.bow = BagOfWords()
        self.class_counts = {}
        self.word_counts = {}
        self.word_probs = {}

    def fit(self, X, y):
        self.classes = np.unique(y)
        
        self.bow.fit(X)
        self.class_counts = {c: 0 for c in self.classes}
        self.word_counts = {c: np.zeros(len(self.bow.vocabulary)) for c in self.classes}
        for sentence, label in zip(X, y):
            vector = self.bow.vectorize(sentence)
            self.class_counts[label] += 1
            self.word_counts[label] += vector


        total_count = sum(self.class_counts.values())
        self.class_priors = {c: count / total_count for c, count in self.class_counts.items()}
        

        self.word_probs = {}
        for c in self.classes:
            smoothed_counts = self.word_counts[c] + 1  # Laplace smoothing
            smoothed_total = sum(self.word_counts[c]) + len(self.bow.vocabulary)
            self.word_probs[c] = smoothed_counts / smoothed_total

        # print("class Priors:", self.class_priors)
        # print("class counts:",self.class_counts)
        # print("word counts:", self.word_counts)
        # print("word probabilities::", self.word_probs)

    def predict(self, X):
        predictions = []
        
        for sentence in X:
            vector = self.bow.vectorize(sentence)
            log_prob = np.log(np.array([self.class_priors[c] for c in self.classes])) 
            
            
            for class_index, c in enumerate(self.classes):
                log_prob[class_index] += np.sum(np.log(self.word_probs[c]) * vector) 
            
            # for c in self.classes:
            #     print("Prob of",c,":",log_prob[c] )
                
            # print("The catgeory will be: ",labels[np.argmax(log_prob)]) 
            
            predictions.append(self.classes[np.argmax(log_prob)])
        
        return predictions


model = MultinomialNaiveBayes()
model.fit(X_train['text'], Y_train)
y_pred = model.predict(X_test['text'])
accuracy = accuracy_score(Y_test, y_pred)
precision = precision_score(Y_test, y_pred, average='macro')
recall = recall_score(Y_test, y_pred, average='macro')
f1 = f1_score(Y_test, y_pred, average='macro')
confusion = confusion_matrix(Y_test, y_pred)

print("Model accuracy:", accuracy)
print("Model precision:", precision)
print("Model recall:", recall)
print("Model F1 score:", f1)
print("Confusion matrix:", confusion)




{'چین': 0, 'ڈرائیور': 1, 'گاڑی': 2, 'پرائمری': 3, 'اسکول': 4, 'طلبہ': 5, 'چڑھا': 6, 'دی': 7, 'زخمی': 8, 'صوبے': 9, 'ہنان': 10, 'راہ': 11, 'گیروں': 12, 'دی۔': 13, 'چینی': 14, 'میڈیا': 15, 'واقعے': 16, 'راہگیر': 17, 'اسپتال': 18, 'منتقل': 19, 'ہے۔': 20, 'برطانوی': 21, 'موجود': 22, 'سیکیورٹی': 23, 'گارڈ': 24, 'والدین': 25, 'پکڑ': 26, 'پولیس': 27, 'حوالے': 28, 'ہے': 29, 'متعلق': 30, 'تفتیش': 31, 'آغاز': 32, 'تیز': 33, 'رفتار': 34, 'کچل': 35, 'دیا': 36, 'نتیجے': 37, 'از': 38, 'ہلاک': 39, 'ہوگئے۔': 40, 'عرب': 41, 'حادثے': 42, 'دیگر': 43, 'شدید': 44, 'ہیں۔': 45, 'رپورٹس': 46, 'وقوع': 47, 'افرا': 48, 'تفری': 49, 'مچ': 50, 'گئی': 51, 'بچوں': 52, 'خوف': 53, 'چیختے': 54, 'دیکھا': 55, 'گیا۔': 56, 'واضح': 57, 'گزشتہ': 58, 'ہفتے': 59, 'تھے۔': 60, 'اسلامی': 61, 'بینکاری': 62, 'مذہبی': 63, 'اسکالرز': 64, 'رائے': 65, 'جائے': 66, 'سینیٹر': 67, 'سلیم': 68, 'مانڈوی': 69, 'سینیٹرسلیم': 70, 'قائمہ': 71, 'کمیٹی': 72, 'برائے': 73, 'خزانہ': 74, 'اجلاس': 75, 'پاکستان': 76, 'کافی': 77, 'کنفیوژن': 78, 'سےمتعلق':